# Introduction to Memory Profiling

> Objectives:
> * Be introduced to memory profiling using different tools
> * Some small introduction to time profiling in IPython too


## ipython_memwatcher

Our recommended way to profile memory consumption for this tutorial will be [ipython_memwatcher](https://pypi.python.org/pypi/ipython_memwatcher):


In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.023 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 31.688 MiB


In [2]:
# Let's create a big object
a = [i for i in range(1000*1000)]

In [2] used 31.129 MiB RAM in 0.074s, peaked 0.000 MiB above current, total RAM usage 62.816 MiB


In [3]:
# Get some measurements from the last executed cell:
meas = mw.measurements
meas

Measurements(memory_delta=31.12890625, time_delta=0.07414698600769043, memory_peak=0, memory_usage=62.81640625)

In [3] used 0.062 MiB RAM in 0.009s, peaked 0.000 MiB above current, total RAM usage 62.879 MiB


In [4]:
# MemWatcher.measurements is a named tuple.  We can easily get info out of it:
meas.memory_delta

31.12890625

In [4] used 0.012 MiB RAM in 0.046s, peaked 0.000 MiB above current, total RAM usage 62.891 MiB


In [5]:
# This takes between 32 ~ 35 bytes per element:
meas.memory_delta * (2**20) / 1e6

32.641024

In [5] used 2.020 MiB RAM in 0.021s, peaked 0.000 MiB above current, total RAM usage 64.910 MiB


In [6]:
# What are these elements made from?
type(a[0])

int

In [6] used 0.027 MiB RAM in 0.011s, peaked 0.000 MiB above current, total RAM usage 64.938 MiB


In [7]:
# How much memory take an int?
# Beware: the size below will depend on whether you are using a 32-bit or 64-bit Python
import sys
sys.getsizeof(a[0])

24

In [7] used 0.027 MiB RAM in 0.030s, peaked 0.000 MiB above current, total RAM usage 64.965 MiB


Ok.  On 64-bits, that means that the int object allocates 8 bytes for the integer value, and 16 bytes for other metadata (Python object).  But 24 is quite less than 32~35.  Where this overhead comes from?

Well, it turns out that the list structure needs additional pointers to reference the different components. [Explain with some diagrams]

## memory_profiler

[memory_profiler](https://pypi.python.org/pypi/memory_profiler) is a basic module for memory profiling that many others use (like the `ipython_memwatcher` above) and it interacts well with ipython, so it is worth to see how it works:

In [8]:
%load_ext memory_profiler

In [8] used 0.035 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 65.000 MiB


In [9]:
# Use %memit magic command exposed by memory_profiler
%memit b = [i for i in range(1000*1000)]

peak memory: 137.08 MiB, increment: 72.08 MiB
In [9] used 72.238 MiB RAM in 0.343s, peaked 0.000 MiB above current, total RAM usage 137.238 MiB


Please note that the `peak_memory` in this case is different than the `peaked_memory` reported by ipython_memwatcher package.

## Guppy

Guppy is nice for having an overview of how different structures are using our memory:

In [10]:
from guppy import hpy; hp=hpy()
hp.heap()

Partition of a set of 2128768 objects. Total size = 81935440 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 2002854  94 48068496  59  48068496  59 int
     1   1233   0 16443528  20  64512024  79 list
     2  58642   3  5120120   6  69632144  85 str
     3  31176   1  2673888   3  72306032  88 tuple
     4   1611   0  1697544   2  74003576  90 dict (no owner)
     5    463   0  1331560   2  75335136  92 dict of module
     6   8024   0  1027072   1  76362208  93 types.CodeType
     7   7865   0   943800   1  77306008  94 function
     8    941   0   847816   1  78153824  95 type
     9    941   0   765752   1  78919576  96 dict of type
<572 more rows. Type e.g. '_.more' to view.>

In [10] used 52.184 MiB RAM in 3.407s, peaked 12.969 MiB above current, total RAM usage 189.422 MiB


In [11]:
del b
hp.heap()

Partition of a set of 1128965 objects. Total size = 49809456 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 1003108  89 24074592  48  24074592  48 int
     1   1222   0  8316272  17  32390864  65 list
     2  58643   5  5120248  10  37511112  75 str
     3  31176   3  2673912   5  40185024  81 tuple
     4   1613   0  1699640   3  41884664  84 dict (no owner)
     5    463   0  1331560   3  43216224  87 dict of module
     6   8024   1  1027072   2  44243296  89 types.CodeType
     7   7864   1   943680   2  45186976  91 function
     8    941   0   847816   2  46034792  92 type
     9    941   0   765752   2  46800544  94 dict of type
<568 more rows. Type e.g. '_.more' to view.>

In [11] used -0.184 MiB RAM in 0.765s, peaked 0.000 MiB above current, total RAM usage 189.238 MiB


In [12]:
# Size of the list (beware, this does not include the contents!)
hp.iso(a)

Partition of a set of 1 object. Total size = 8126536 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1 100  8126536 100   8126536 100 list

In [12] used -0.016 MiB RAM in 0.106s, peaked 0.000 MiB above current, total RAM usage 189.223 MiB


## %time and %timeit

In [13]:
# IPython provides a magic command to see how much time a command takes to run
%time asum = sum(a)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 11.8 ms
In [13] used 0.000 MiB RAM in 0.019s, peaked 0.000 MiB above current, total RAM usage 189.223 MiB


Note that `%time` offers quite detailed statistics on the time spent.

Also, the time reported by MemoryWatcher has a typical overhead of 3~10 ms over the time reported by %time, so when the times to measure are about this order then it is better to rely on the %time (or %timeit below) values.  

In [14]:
# We have another way to measure timings doing several loops and getting the mean
%timeit bsum = sum(a)
# However, one must notice that %timeit does not return the result of expressions

100 loops, best of 3: 6.92 ms per loop
In [14] used 0.035 MiB RAM in 2.937s, peaked 0.000 MiB above current, total RAM usage 189.258 MiB


Interestingly, %timeit allows to retrieve the measured times in loops with the -o flag:

In [15]:
t = %timeit -o sum(a)
print(t.all_runs)
print(t.best)

100 loops, best of 3: 7 ms per loop
[0.7001528739929199, 0.7139549255371094, 0.7227981090545654]
0.00700152873993
In [15] used 0.004 MiB RAM in 3.036s, peaked 0.000 MiB above current, total RAM usage 189.262 MiB


And one can specify the number of loops (-n) and the number of repetitions (-r):

In [16]:
t = %timeit -r1 -n1 -o sum(a)
print(t.all_runs)
print(t.best)

1 loops, best of 1: 14.5 ms per loop
[0.014538049697875977]
0.0145380496979
In [16] used 0.000 MiB RAM in 0.112s, peaked 0.000 MiB above current, total RAM usage 189.262 MiB


### Exercise 1

Provided a dictionary like:

```
d = dict(("key: %i"%i, i*2) for i in a)
```

Try to guess how much RAM it uses using the techniques introduced above.

Why do you think it takes more space than a list?

*Hint*: Every entry in a dictionary has pointers to two objects: key and value. 

## Solution

In [17]:
d = dict(("key: %i"%i, i*2) for i in a)

In [17] used 100.930 MiB RAM in 0.936s, peaked 0.000 MiB above current, total RAM usage 290.191 MiB


In [18]:
# Compute the size of key + value
sys.getsizeof("key: 100000") + sys.getsizeof(1)

72

In [18] used 0.012 MiB RAM in 0.004s, peaked 0.000 MiB above current, total RAM usage 290.203 MiB


In [19]:
# Compute the size of a key + value + pointers for all elements in MB
(sys.getsizeof("key: 100000") + sys.getsizeof(1) + 16) * 1000*1000 / 2**20.

83.92333984375

In [19] used 0.000 MiB RAM in 0.006s, peaked 0.000 MiB above current, total RAM usage 290.203 MiB


In [20]:
# Using guppy
hp.iso(d)

Partition of a set of 1 object. Total size = 50331928 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1 100 50331928 100  50331928 100 dict (no owner)

In [20] used 0.008 MiB RAM in 0.586s, peaked 0.000 MiB above current, total RAM usage 290.211 MiB


So, guppy is telling us that just the dictionary structure is taking ~50 MB, whereas the contents alone are taking ~70MB, so we should have expected the dictionary to consume ~120 MB.  However, our `MemWatcher` instance is reporting just ~100 MB.  Take away lesson: measuring memory consumption in Python is tricky but using proper tools we can still get valuable hints!